This notebook applies to drx+tet structures such as the partially disordered spinel where Li+, Mn2+, and Vacancies occupy octahedral and tetrahedral sublattices. 

Authors: Julia Yang, Tina Chen

Date: May 2022

## 1. Load all structure data

In [11]:
import json
import numpy as np

starting_spinel_file = 'data/limn2o4_spinel.json'
mson = 'data/lmof_drx_tet.mson'

## 2. Load a trained cluster expansion model

In [12]:
from smol.io import load_work
from smol.cofe.expansion import ClusterExpansion 

# load the starting configuration which is a spinel structure
from pymatgen.core.structure import *
with open(starting_spinel_file, 'r') as f: 
    init_struct = Structure.from_dict(json.load(f))

# this is the supercell shape of the spinel structure
m = np.matrix([[2,2,-2],[-2,2,2],[2,-2,2]]) 

# run a small cell for now 
transformation = np.identity(3)
supercell_matrix = m @ transformation

# load CE
#work = load_work(mson)
#expansion = work['ClusterExpansion']
with open(mson, 'r') as f: expansion = ClusterExpansion.from_dict(json.load(f))

AttributeError: The size of the give subspace 175 does not match the number of coefficients 174

In [10]:
print (expansion['feature_matrix'])
print (expansion['coefs'].shape)

None


AttributeError: 'list' object has no attribute 'shape'

## 3. Create the Ensemble and use in MC sampling 

If you want to save the occupancies for later analysis of 16c or 16d sites, you *must* save current_occupancy now, as generating this processor via `ensemble.processor` does not maintain order. In other words, the 16c and 16d sites index to different indices if you analyze the 16c or 16d sites from generating another processor later, say processor2 for clarity, via `processor2.occupancy_from_structure(init_struct)`. This is not necessarily a bug. Just be aware that one must exercise caution when comparing bit-by-bit. 

`Mn_swap_probability` is the probability of choosing a flip from `Mn_flip_table` which contain disproportionation reactions possible in Mn-oxides. See John Reed's thesis for more information.

In [19]:
Mn_flip_table = {('Mn2+', 'Mn2+'): ['None'],
                 ('Mn2+', 'Mn3+'): ['swap'],
                 ('Mn3+', 'Mn2+'): ['swap'],
                 ('Mn2+', 'Mn4+'): ['dispropA', 'swap'],
                 ('Mn4+', 'Mn2+'): ['dispropB', 'swap'],
                 ('Mn3+', 'Mn3+'): ['dispropC', 'dispropD'],
                 ('Mn3+', 'Mn4+'): ['swap'],
                 ('Mn4+', 'Mn3+'): ['swap'],
                 ('Mn4+', 'Mn4+'): ['None']}

In [20]:
from smol.moca import Sampler 
from smol.moca.ensemble.canonical import CanonicalEnsemble

ALLOW_CROSSOVER=True
STEP_TYPE='tableswapper'
NUMBER_OF_PROPOSALS=20 # quick T scan parameter 
START_T = 300 
END_T = 400 # quick T scan parameter 
STEP_T = 100

#ensemble = CanonicalEnsemble.from_cluster_expansion(expansion, 
#                                                    supercell_matrix)
current_occupancy = ensemble.processor.occupancy_from_structure(init_struct)

for T in range(START_T, END_T+1, STEP_T): 
    sampler = Sampler.from_ensemble(kernel_type='Metropolis',
                                    ensemble=ensemble,
                                    temperature=T,
                                    step_type=STEP_TYPE,
                                    allow_cross_over=True,
                                    Mn_swap_probability=0.5,
                                    swap_table=None, 
                                    Mn_flip_table=Mn_flip_table
                                )
    sampler.run(NUMBER_OF_PROPOSALS,
               initial_occupancies=current_occupancy)

ValueError: Step type tableswapper can not be used for sampling a <class 'smol.moca.ensemble.canonical.CanonicalEnsemble'>!

In [26]:
from smol.cofe import ClusterSubspace 
import numpy as np
supercell_matrix= np.matrix([[2,2,-2],[-2,2,2],[2,-2,2]]) 

prim = Structure.from_file("../../20181127_newPrim.cif")
#prim.remove_sites([0, 2])
prim.replace_species({"Cs": {'Li+': 0.5}, #tet
                      "Rb": {'Li+': 0.5}, #tet # this domain really matters!!
                      "K": {'Li+': 0.2, 'Mn3+': 0.2, 'Mn4+':0.2},  #oct
                      "Cl": {'O2-': 1.0}})
cs = ClusterSubspace.from_cutoffs(structure=prim,
                                 cutoffs={2:3},# 4588 ECI + Ewald
                                 ltol = 0.15, stol = 0.2, angle_tol = 15,
                                 supercell_size=('O2-', 'F-'),
                                 basis='sinusoid',
                                 orthonormal=False)
coef=np.random.rand(len(cs))
from smol.moca.processor.base import Processor
CanonicalEnsemble(Processor(cs, supercell_matrix, coef))

TypeError: Can't instantiate abstract class Processor with abstract methods compute_feature_vector, compute_feature_vector_change

In [27]:
returnd = ensemble.as_dict()

In [29]:
with open('data/canonical_ensemble_example.json','w') as fout:
    json.dump(returnd, fout)

KeyboardInterrupt: 